# VAE Training

## imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os

from models.VAE import VariationalAutoencoder
from utils.loaders import load_mnist

import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.75
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
# run params
SECTION = 'vae'
RUN_ID = '0002'
DATA_NAME = 'digits'
RUN_FOLDER = 'run/{}/'.format(SECTION)
RUN_FOLDER += '_'.join([RUN_ID, DATA_NAME])

if not os.path.exists(RUN_FOLDER):
    os.mkdir(RUN_FOLDER)
    os.mkdir(os.path.join(RUN_FOLDER, 'viz'))
    os.mkdir(os.path.join(RUN_FOLDER, 'images'))
    os.mkdir(os.path.join(RUN_FOLDER, 'weights'))

mode =  'build' #'load' #

## data

In [4]:
(x_train, y_train), (x_test, y_test) = load_mnist()

## architecture

In [5]:
vae = VariationalAutoencoder(
    input_dim = (28,28,1)
    , encoder_conv_filters = [32,64,64, 64]
    , encoder_conv_kernel_size = [3,3,3,3]
    , encoder_conv_strides = [1,2,2,1]
    , decoder_conv_t_filters = [64,64,32,1]
    , decoder_conv_t_kernel_size = [3,3,3,3]
    , decoder_conv_t_strides = [1,2,2,1]
    , z_dim = 2
    , r_loss_factor = 1000
)

if mode == 'build':
    vae.save(RUN_FOLDER)
else:
    vae.load_weights(os.path.join(RUN_FOLDER, 'weights/weights.h5'))

In [6]:
vae.encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
encoder_conv_0 (Conv2D)         (None, 28, 28, 32)   320         encoder_input[0][0]              
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 28, 28, 32)   0           encoder_conv_0[0][0]             
__________________________________________________________________________________________________
encoder_conv_1 (Conv2D)         (None, 14, 14, 64)   18496       leaky_re_lu[0][0]                
____________________________________________________________________________________________

In [7]:
vae.decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
decoder_input (InputLayer)   [(None, 2)]               0         
_________________________________________________________________
dense (Dense)                (None, 3136)              9408      
_________________________________________________________________
reshape (Reshape)            (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_0 (Conv2DTran (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
decoder_conv_t_1 (Conv2DTran (None, 14, 14, 64)        36928     
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 14, 14, 64)        0   

## training

In [8]:
LEARNING_RATE = 0.0005

In [9]:
vae.compile(LEARNING_RATE)

In [10]:
BATCH_SIZE = 32
EPOCHS = 200
PRINT_EVERY_N_BATCHES = 100
INITIAL_EPOCH = 0

In [ ]:
vae.train(     
    x_train
    , batch_size = BATCH_SIZE
    , epochs = EPOCHS
    , run_folder = RUN_FOLDER
    , print_every_n_batches = PRINT_EVERY_N_BATCHES
    , initial_epoch = INITIAL_EPOCH
)

run/vae/0002_digits
Epoch 1/200
1873/1875 [============================>.] - ETA: 0s - loss: 58.4865 - reconstruction_loss: 55.2373 - kl_loss: 3.2492
Epoch 00001: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 19s 10ms/step - loss: 58.4786 - reconstruction_loss: 55.2285 - kl_loss: 3.2501 - lr: 5.0000e-04
Epoch 2/200
1874/1875 [============================>.] - ETA: 0s - loss: 51.5809 - reconstruction_loss: 47.5564 - kl_loss: 4.0245
Epoch 00002: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 19s 10ms/step - loss: 51.5792 - reconstruction_loss: 47.5543 - kl_loss: 4.0249 - lr: 5.0000e-04
Epoch 3/200
1871/1875 [============================>.] - ETA: 0s - loss: 49.9675 - reconstruction_loss: 45.6428 - kl_loss: 4.3247
Epoch 00003: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 49.9707 - reconstruction_loss: 45.6458

1871/1875 [============================>.] - ETA: 0s - loss: 45.4482 - reconstruction_loss: 40.3550 - kl_loss: 5.0933
Epoch 00025: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 45.4433 - reconstruction_loss: 40.3495 - kl_loss: 5.0938 - lr: 5.0000e-04
Epoch 26/200
1873/1875 [============================>.] - ETA: 0s - loss: 45.4093 - reconstruction_loss: 40.3008 - kl_loss: 5.1085
Epoch 00026: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 45.4130 - reconstruction_loss: 40.3047 - kl_loss: 5.1083 - lr: 5.0000e-04
Epoch 27/200
1872/1875 [============================>.] - ETA: 0s - loss: 45.3815 - reconstruction_loss: 40.2630 - kl_loss: 5.1185
Epoch 00027: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 45.3892 - reconstruction_loss: 40.2702 - kl_loss: 5.1190 - lr: 5.0000e

1875/1875 [==============================] - ETA: 0s - loss: 44.6434 - reconstruction_loss: 39.4135 - kl_loss: 5.2299
Epoch 00049: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 44.6426 - reconstruction_loss: 39.4128 - kl_loss: 5.2299 - lr: 5.0000e-04
Epoch 50/200
1872/1875 [============================>.] - ETA: 0s - loss: 44.6068 - reconstruction_loss: 39.3721 - kl_loss: 5.2348
Epoch 00050: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 44.6030 - reconstruction_loss: 39.3685 - kl_loss: 5.2345 - lr: 5.0000e-04
Epoch 51/200
1871/1875 [============================>.] - ETA: 0s - loss: 44.5728 - reconstruction_loss: 39.3324 - kl_loss: 5.2404
Epoch 00051: saving model to run/vae/0002_digits/weights/weights.h5
1875/1875 [==============================] - 17s 9ms/step - loss: 44.5704 - reconstruction_loss: 39.3300 - kl_loss: 5.2404 - lr: 5.0000e

In [ ]:
print(tf.__version__)